In [29]:
AmerPutLSM_CV <-
  function (Spot=1, sigma=0.2, n=1000, m=365, Strike=1.1, r=0.06, dr=0.0, mT=1){
    GBM<-matrix(NA, nrow=n, ncol=m)
    for(i in 1:n) {
      GBM[i,]<-Spot*exp(cumsum(((r-dr)*(mT/m)-0.5*sigma*sigma*(mT/m))+(sigma*(sqrt(mT/m))*rnorm(m, mean= 0, sd=1))))
    }
    X<-ifelse(GBM<Strike,GBM,NA)
    CFL<-matrix(pmax(0,Strike-GBM), nrow=n, ncol=m)
    Xsh<-X[,-m]
    X2sh<-Xsh*Xsh
    Y1<-CFL*exp(-1*r*(mT/m))
    Y2<-cbind((matrix(NA, nrow=n, ncol=m-1)), Y1[,m])
    CV<-matrix(NA, nrow=n, ncol=m-1)
    try(for(i in (m-1):1) {
      reg1<-lm(Y2[,i+1]~Xsh[,i]+X2sh[,i])
      CV[,i]<-(matrix(reg1$coefficients)[1,1])+((matrix(reg1$coefficients)[2,1])*Xsh[,i])+((matrix(reg1$coefficients)[3,1])*X2sh[,i])
      CV[,i]<-(ifelse(is.na(CV[,i]),0,CV[,i]))
      Y2[,i]<-ifelse(CFL[,i]>CV[,i], Y1[,i], Y2[,i+1]*exp(-1*r*(mT/m)))
    }
    , silent = TRUE)
    CV<-ifelse(is.na(CV),0,CV)
    CVp<-cbind(CV, (matrix(0, nrow=n, ncol=1)))
    POF<-ifelse(CVp>CFL,0,CFL)
    FPOF<-firstValueRow(POF)
    dFPOF<-matrix(NA, nrow=n, ncol=m)
    for(i in 1:m) {
      dFPOF[,i]<-FPOF[,i]*exp(-1*mT/m*r*i)
    }
    PRICE1<-mean(rowSums(dFPOF))
    dCFL<-CFL[,m]*exp(-1*r*mT)
    EUsimul<-mean(dCFL)
    EUBS<-EuPutBS(Spot,sigma,Strike,r,dr,mT)
    PRICE<-PRICE1-(EUsimul-EUBS)
    res<- list(price=(PRICE), Spot, Strike, sigma, n, m, r, dr, mT)
    class(res)<-"AmerPutCV"
    return(res)
  }

firstValueRow <-
  function(x) {
    cumSumMat<-matrix(NA, nrow=dim(x)[1], ncol=dim(x)[2])
    for(i in 1:(dim(x)[1])) {
      cumSumMat[i,]<-cumsum(x[i,])
    }
    cumSumMat2<-cbind(matrix(0, nrow=dim(x)[1], ncol=1), cumSumMat[,-(dim(x)[2])])
    ResultMat<-matrix(NA, nrow=dim(x)[1], ncol=dim(x)[2])
    for(i in 1:dim(x)[2]) {
      ResultMat[,i]<-ifelse(cumSumMat2[,i]>0, 0, x[,i])
    }
    return(ResultMat)	
  }

EuPutBS <-
  function (Spot,sigma,Strike,r,dr,mT){
    forward<-Spot*exp((r-dr)*mT)
    dplus<-(log(forward/Strike)+0.5*sigma*sigma*mT)/(sigma*sqrt(mT))
    dminus<-(log(forward/Strike)-0.5*sigma*sigma*mT)/(sigma*sqrt(mT))
    mdplus<-dplus*(-1)
    mdminus<-dminus*(-1)
    Ndpl<-pnorm(dplus)
    Ndmin<-pnorm(dminus)
    Nmdpl<-pnorm(mdplus)
    Nmdmin<-pnorm(mdminus)
    V0_BS<-exp(-1*r*mT)*(forward*Ndpl-Strike*Ndmin)
    V0P_BS<-exp(-1*r*mT)*(Strike*Nmdmin-forward*Nmdpl)
    return(V0P_BS)
  }
put=AmerPutLSM_CV(Spot=1, sigma=0.2, n=1000, m=365, Strike=1.1, r=0.06, dr=0.0, mT=1)
put$price

[1] 0.1137606

In [3]:
Spot=1
sigma=0.2
n=1000
m=365
Strike=1.1
r=0.06
dr=0.0
mT=1

In [83]:
set.seed(5)
GBM<-matrix(NA, nrow=n, ncol=m)
for(i in 1:n) {
  GBM[i,]<-Spot*exp(cumsum(((r-dr)*(mT/m)-0.5*sigma*sigma*(mT/m))+(sigma*(sqrt(mT/m))*rnorm(m, mean= 0, sd=1))))
}
X<-ifelse(GBM<Strike,GBM,NA)
CFL<-matrix(pmax(0,Strike-GBM), nrow=n, ncol=m)
Xsh<- X[,-m] # exclude the last column
X2sh<- Xsh*Xsh
Y1<-CFL*exp(-1*r*(mT/m))
Y2<-cbind((matrix(NA, nrow=n, ncol=m-1)), Y1[,m])
CV<-matrix(NA, nrow=n, ncol=m-1)
try(for(i in (m-1):1) {
  reg1<-lm(Y2[,i+1]~Xsh[,i]+X2sh[,i])
  CV[,i]<-(matrix(reg1$coefficients)[1,1])+((matrix(reg1$coefficients)[2,1])*Xsh[,i])+((matrix(reg1$coefficients)[3,1])*X2sh[,i])
  CV[,i]<-(ifelse(is.na(CV[,i]),0,CV[,i]))
  Y2[,i]<-ifelse(CFL[,i]>CV[,i], Y1[,i], Y2[,i+1]*exp(-1*r*(mT/m)))
}, silent = TRUE)

In [71]:
CV<-ifelse(is.na(CV),0,CV)
CVp<-cbind(CV, (matrix(0, nrow=n, ncol=1)))
POF<-ifelse(CVp>CFL,0,CFL)
FPOF<-firstValueRow(POF)
dFPOF<-matrix(NA, nrow=n, ncol=m)
for(i in 1:m) {
  dFPOF[,i]<-FPOF[,i]*exp(-1*mT/m*r*i)
}
PRICE1<-mean(rowSums(dFPOF))
dCFL<-CFL[,m]*exp(-1*r*mT)
EUsimul<-mean(dCFL)
EUBS<-EuPutBS(Spot,sigma,Strike,r,dr,mT)
PRICE<- PRICE1-(EUsimul-EUBS) #???
res<- list(price=(PRICE), Spot, Strike, sigma, n, m, r, dr, mT)
class(res)<-"AmerPutCV"

In [84]:
CV<-ifelse(is.na(CV),0,CV)
CVp<-cbind(CV, (matrix(0, nrow=n, ncol=1)))

In [85]:
POF<-ifelse(CVp>CFL,0,CFL)
FPOF<-firstValueRow(POF)

In [91]:
mean(Y2[,1])

[1] 0.1197818

In [92]:
PRICE1

[1] 0.1197818

In [87]:
FPOF[1,]

[1] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
  [8] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [15] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [22] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [29] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [36] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [43] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [50] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [57] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [64] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [71] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [78] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [85] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [92] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [99] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[106] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[113] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[120] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[127] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[134] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[141] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[148] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[155] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[162] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[169] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[176] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[183] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[190] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[197] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[204] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[211] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[218] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[225] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[232] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[239] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[246] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[253] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[260] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[267] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[274] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[281] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[288] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[295] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[302] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[309] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[316] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[323] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[330] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[337] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[344] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[351] 0.1323224 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[358] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
[365] 0.0000000

In [21]:
a=Spot*exp(cumsum(((r-dr)*(mT/m)-0.5*sigma*sigma*(mT/m))+(sigma*(sqrt(mT/m))*rnorm(m, mean= 0, sd=1))))

In [54]:
Y1<-CFL*exp(-1*r*(mT/m))
Y2<-cbind((matrix(NA, nrow=n, ncol=m-1)), Y1[,m])

In [59]:
reg1<-lm(Y2[,364+1]~Xsh[,364]+X2sh[,364])

In [66]:
i=364

In [67]:
CV[,i]<-(matrix(reg1$coefficients)[1,1])+((matrix(reg1$coefficients)[2,1])*Xsh[,i])+((matrix(reg1$coefficients)[3,1])*X2sh[,i])

In [69]:
CV[,i]

[1] 0.0876622786           NA 0.0584473992           NA           NA
   [6]           NA           NA 0.3369837831           NA 0.2453362102
  [11] 0.0837426591 0.0679286671           NA           NA 0.1872407923
  [16] 0.1251626380 0.2484217749           NA 0.1522430586 0.4022978002
  [21] 0.0941323073 0.3854988348 0.1752561558           NA           NA
  [26]           NA 0.2508626535           NA 0.2759974990 0.4194836698
  [31]           NA 0.4209003878 0.3384167189 0.1862993153 0.2326393009
  [36] 0.0490156334           NA 0.2274856951           NA 0.3786967496
  [41]           NA           NA 0.1627296815 0.3204749698 0.1220963006
  [46] 0.0269849225 0.1887445782 0.0254357277           NA 0.0820339362
  [51]           NA 0.2320794678 0.1857889822           NA 0.3871279881
  [56]           NA 0.0727165748           NA 0.1640229953 0.2205460698
  [61] 0.2895896892           NA 0.2131923090           NA 0.0544103228
  [66] 0.1280238359 0.2770386383 0.1269444225           NA 0.2707094212
  [71] 0.1986920300           NA           NA           NA 0.0177386785
  [76] 0.4130491160 0.3481874664           NA 0.2473221536 0.1086041738
  [81] 0.0058081643           NA           NA 0.0893203052           NA
  [86] 0.0852453615           NA 0.1741974791 0.1191614464           NA
  [91]           NA           NA           NA 0.3202145847 0.1121151496
  [96]           NA 0.0746081591 0.4714889911 0.1529021188           NA
 [101] 0.0675171153           NA 0.1564279067 0.3718292409           NA
 [106] 0.3638853023 0.0011475004 0.2265444779 0.1137324125 0.0621066838
 [111] 0.1586911023           NA 0.3156586829           NA 0.0948413698
 [116]           NA 0.1513323225 0.2605629275           NA 0.3093970650
 [121]           NA 0.0752190953           NA 0.1203360603 0.1400764545
 [126]           NA 0.3475300601           NA           NA 0.0629485746
 [131]           NA 0.3592284986           NA 0.2135241490 0.1279289833
 [136] 0.0642212275           NA           NA 0.0976934726 0.4459445827
 [141]           NA 0.2722445543           NA           NA           NA
 [146]           NA           NA 0.0088432614 0.1997669605 0.2070751427
 [151] 0.1354402849 0.3135990090 0.2262260888           NA 0.1496203704
 [156] 0.1890791285           NA 0.2253640737 0.3499131390 0.3819679284
 [161]           NA 0.2670306886           NA 0.1154253219 0.0677664669
 [166] 0.0330889563 0.3184323110 0.4767091988 0.2661306508 0.0335156638
 [171] 0.0750176964 0.3914375595           NA 0.0795669359 0.0028800016
 [176] 0.1704000531 0.2341923273 0.3345099845           NA 0.1795506015
 [181] 0.2760132390           NA 0.3714465258 0.1102628272           NA
 [186] 0.2431826343 0.0379196728           NA           NA           NA
 [191] 0.1273292558           NA 0.0928149398 0.2016654918 0.2650657389
 [196]           NA 0.3532707505 0.1091368955 0.1061537642 0.3222758319
 [201] 0.1254408282 0.2288766050 0.2218222852 0.1360755132 0.0734667031
 [206] 0.2083381093           NA           NA           NA 0.1891060741
 [211]           NA 0.0876346241 0.4248031812 0.0732918998 0.2937472278
 [216]           NA 0.3614230542 0.1133364890 0.1073248794           NA
 [221]           NA           NA           NA 0.0604739172 0.3074426130
 [226] 0.1687295516 0.2926369489 0.0847510860 0.0899497340 0.2244605768
 [231] 0.2608264311 0.3706056606           NA 0.1188383843 0.1678344355
 [236] 0.0807872076 0.2052114119 0.3072094906 0.2532504876 0.2837780457
 [241]           NA           NA 0.2824785999 0.0145807006 0.1991379947
 [246]           NA 0.0788770247 0.1324093355 0.1212774600           NA
 [251] 0.1563293780 0.2546871025 0.3336959583           NA 0.6324567830
 [256] 0.0422021905           NA           NA 0.2921053976           NA
 [261]           NA 0.1219042401           NA           NA           NA
 [266]           NA           NA           NA 0.2510999605 0.1288683752
 [271] 0.0593521983 0.0937742847 0.2992669684 0.0057557009 0.2065377905
 [276]           NA 0.1937874951 0.19602409